In [ ]:
%%writefile Crawling_Utils.py
import numpy as np
import urllib.request
from bs4 import BeautifulSoup
import re
import json
from datetime import datetime

def getSoup(url, parser):
    hdr = {'User-Agent':'Mozilla/5.0'}
    req = urllib.request.Request(url,headers=hdr)
    with urllib.request.urlopen(req) as url:
        doc = url.read()
    return BeautifulSoup(doc, parser)

# def getMetaData(url):
    

def getOkList(soup, isPC = 1):
    okListTime = []
    p = re.compile('\d\d\d\d/\d\d/\d\d \d\d:\d\d:\d\d')
    if isPC:
        okList = p.findall(soup.find_all("script")[0].text) #PC:0, Mobile:-1
    else:
        okList = p.findall(soup.find_all("script")[-1].text) #PC:0, Mobile:-1
    okListTime = list(map(lambda x: datetime.strptime(x, '%Y/%m/%d %H:%M:%S'), okList ))
    return okListTime
  


def getMemoList(soup):
    script = soup.find_all("script")
    p = re.compile('parent_table = ".*?"')
    p = p.search(script[0].text)
    parent_table = script[0].text[p.start():p.end()]
    parent_table = parent_table[16:-1]

    p = re.compile('parent_id = ".*?"')
    p = p.search(script[0].text)
    parent_id = script[0].text[p.start():p.end()]
    parent_id = parent_id[13:-1]
    memo_url = "/board/ajax_memo_list.php?parent_table=" + parent_table + "&parent_id=" + parent_id;
    memo_url = 'http://www.todayhumor.co.kr' + memo_url
#     print(memo_url)

    memo_soup =  getSoup(memo_url, "html.parser")
#     memoData = dict()
    memoList = []
    memos = json.loads(memo_soup.text)['memos']
    bestTime = 0
    bObTime = 0
    for memo in memos:
        if 'MOVE_HUMORBEST' in memo['memo']:
            bestTime = datetime.strptime(memo['date'], '%Y-%m-%d %H:%M:%S')
        elif 'MOVE_BESTOFBEST' in memo['memo']:
            bObTime = datetime.strptime(memo['date'], '%Y-%m-%d %H:%M:%S')
        else:
            date = datetime.strptime(memo['date'], '%Y-%m-%d %H:%M:%S')
            memoList.append(date)
#             date = memo['date']
#             memoData[date] = [int(memo['ok']), int(memo['nok'])]
    return memoList
#     json.loads(memo_soup.text)['memos']

In [ ]:
url = "http://www.todayhumor.co.kr/board/view.php?table=bestofbest&no=288310&s_no=288310&page=1"

soup = getSoup(url, "html.parser")
okList = getOkList(soup)
(bTime,bobTime,memoList) = getMemoList(soup)

writerInfoContents = soup.find_all("div", class_="writerInfoContents")
divTags = writerInfoContents[0].find_all("div")
if len(divTags) is 10:
    postTime = datetime.strptime(divTags[7].text.split(' : ')[1], '%Y/%m/%d %H:%M:%S')  # 게시 시간
else:
    postTime = datetime.strptime(divTags[6].text.split(' : ')[1], '%Y/%m/%d %H:%M:%S')  # 게시 시간
    
ok_time = list(map(lambda x: (x - postTime).total_seconds()/3600.0, okList[:50]))
memo_time = list(map(lambda x: (x - postTime).total_seconds()/3600.0, memoList))

In [ ]:
len(divTags)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

fig1, (ax1) = plt.subplots(nrows=1, ncols=1, sharex=False)
ax1.plot(ok_time, np.arange(1, len(ok_time)+1), 'g+-', label='ok', linewidth=1)
ax1.set_ylabel('ok', color='g')
ax1.tick_params(axis='x', colors='w')
ax1.tick_params(axis='y', colors='g')
ax1.set_xlabel('time(hour)')

ax2 = ax1.twinx() 
ax2.plot(memo_time, np.arange(1, len(memo_time)+1), 'rx-', label='ok', linewidth=1)
ax2.set_ylabel('memo', color='r')
ax2.tick_params(axis='y', colors='red')
ax2.set_xlim(0, ok_time[-1])

In [ ]:
ax2.yaxis.get_ticklabels()

In [ ]:
import numpy as np
article_no = np.random.randint(250000,288325, size=10)

dbForOk = dict()
for i in article_no:
    url = 'http://m.todayhumor.co.kr/view.php?table=bestofbest&no={}'.format(i)
    print(url)
    soup = getSoup(url, 'html.parser')
#     if not len(soup.find_all('title')): continue
    okList = getOkList(soup, isPC = 0)
    postTime = datetime.strptime(soup.find_all('span', class_="view_wdate")[0].text, '%Y/%m/%d %H:%M:%S')
    okTime = list(map(lambda x: (x - postTime).total_seconds(), okList))
#     <span class="view_wdate">2016/12/08 22:41:06</span>
    dbForOk[i] = okTime



In [ ]:
import pickle as pkl
with open('db_bob_oks.pkl', 'wb') as f:
    pkl.dump(dbForOk, f)

In [5]:
import pickle as pkl
with open('sisa_db.pkl', 'rb') as f:
    a = pkl.load(f)

In [ ]:
getSoup('http://m.todayhumor.co.kr/view.php?table=humorbest&no=1348480&page=1', 'html.parser')
viewSubjectDiv = soup.find_all("div")
viewSubjectDiv

In [8]:
import pandas as pd

pd.DataFrame(a)

,contentID,writerName,writerSignInDate,writerVisitingCount,recommendCount,viewCount,memoCount,postTime,normalPostCount,bestPostCount,...,prohibitBestOption,prohibitBOBOption,prohibitBoninOption,prohibitOutsidePermOption,imgCount,videoCount,youtubeCount,textLineCount,okTime,memoTime
0,sisa_813074,구찌입니다,2016-03-22,240,36,455,24,2016-12-09 17:11:34,462,78,...,False,False,False,False,0,0,0,0,"[46.0, 52.0, 121.0, 138.0, 167.0, 170.0, 215.0...","[270.0, 529.0, 570.0, 592.0, 597.0, 652.0, 667..."
1,sisa_813151,어쩌다알통,2016-08-08,31,9,1163,1,2016-12-09 17:32:18,57,10,...,False,False,False,False,0,0,0,0,"[31.0, 43.0, 53.0, 121.0, 186.0, 232.0, 307.0,...",[184.0]
2,sisa_813545,헛참,2011-11-12,2642,9,648,0,2016-12-09 20:21:56,2,0,...,False,False,False,False,0,0,1,0,"[184.0, 401.0, 450.0, 607.0, 626.0, 3127.0, 32...",[]
3,sisa_813183,퇴개미,2016-08-28,92,6,182,2,2016-12-09 17:41:08,1,0,...,False,False,False,False,0,0,0,10,"[69.0, 74.0, 450.0, 954.0, 5406.0, 15665.0]","[113.0, 1252.0]"
4,sisa_813536,KBK,2011-09-03,551,4,178,0,2016-12-09 20:15:56,51,4,...,False,False,False,False,0,0,0,0,"[6.0, 225.0, 324.0, 5549.0]",[]
5,sisa_813225,원조잡초,2010-07-27,489,16,431,1,2016-12-09 17:51:30,371,112,...,False,False,False,False,0,0,0,7,"[71.0, 84.0, 117.0, 121.0, 161.0, 186.0, 204.0...",[334.0]
6,sisa_813789,심플맨,2015-03-14,127,5,633,2,2016-12-09 22:20:31,240,35,...,False,False,False,False,0,0,0,0,"[105.0, 1120.0, 1458.0, 2133.0, 2411.0]","[351.0, 655.0, 891.0]"
7,sisa_813719,열받으면짖음,2005-07-26,507,0,278,0,2016-12-09 21:35:20,32,23,...,False,False,False,False,0,0,0,0,[],[]
8,sisa_813434,크롱♥,2013-06-10,1165,1,1015,4,2016-12-09 19:13:12,24,13,...,False,False,False,True,3,0,0,3,[759.0],"[143.0, 155.0, 168.0, 251.0]"


In [ ]:
url = 'http://m.todayhumor.co.kr/view.php?table=humorbest&no=1348480&page=1'
hdr = {'User-Agent':'Mozilla/5.0'}
proxies = {'html', 'http://deng.in.net'}
req = urllib.request.Request(url,headers=hdr)
doc = ""
with urllib.request.urlopen(req, ) as url:
    doc = url.read()

soup = BeautifulSoup(doc, "html.parser")


In [2]:
%%writefile getArticleInfos.py
import urllib.request
from bs4 import BeautifulSoup
from datetime import datetime
def getArticleInfos(url):
    infos = ['contentID', 'writerName', 'writerSignInDate', 'writerSignInDate', 'writerVisitingCount',
            'recommendCount', 'viewCount', 'memoCount' , 'postTime', 'normalPostCount', 'bestPostCount', 'BOBPostCount',
            'board', 'title', 'changjakOption', 'permOption', 'prohibitBestOption', 'prohibitBOBOption', 'prohibitBoninOption', 
            'prohibitOutsidePermOption', 'imgCount', 'videoCount',  'youtubeCount', 'textLineCount', 
            'okTime', 'memoTime' ]

    hdr = {'User-Agent':'Mozilla/5.0'}
    req = urllib.request.Request(url,headers=hdr)

    doc = ""
    with urllib.request.urlopen(req) as url:
        doc = url.read()

    soup = BeautifulSoup(doc, "html.parser")

    ## get Writer Infomation
    writerInfoContents = soup.find_all("div", class_="writerInfoContents")
    divTags = writerInfoContents[0].find_all("div")

    contentID = divTags[0].text.split(' : ')[1]
    writerName = (divTags[1].find_all("b"))[0].text   # 글쓴이 닉네임
    writerInfo_ = (divTags[1].find_all("span"))[-1].text
    writerSignInDate = re.split(':| |\)',writerInfo_)[1]   # 글쓴이 가입 날짜
    writerVisitingCount = re.split(':| |\)',writerInfo_)[3]  # 방문 횟수
    recommendCount = divTags[2].find_all("span")[0].text   # 추천수
    viewCount = divTags[3].text.split(' : ')[1]            # 조회수 
    memoCount = re.split(' : |개',divTags[5].text)[1]   # 댓글 수 
    # BOBTime = divTags[6].text.split(' : ')[1]              # 베스트 등록 시간
    postTime = divTags[6].text.split(' : ')[1]             # 게시 시간

    #### 게시자의 다른 글 개수 ####
    baseURL = "http://www.todayhumor.co.kr/board/"
    writerProfileURLSuffix = divTags[1].find_all("a")[0]["href"]
    writerProfileURL = baseURL + writerProfileURLSuffix
    writerProfileReq = urllib.request.Request(writerProfileURL,headers=hdr)

    writerProfileDoc = ""
    with urllib.request.urlopen(writerProfileReq) as url:
        writerProfileDoc = url.read()

    writerProfileSoup = BeautifulSoup(writerProfileDoc, "html.parser")    
    table_list = writerProfileSoup.find_all("table", class_="table_list")
    normalPostCount = table_list[0].find("a").text         # 유저가 쓴 글 개수

    baseURL = "http://www.todayhumor.co.kr"
    member_menu_box = writerProfileSoup.find_all("div", class_="member_menu_box")
    writerProfileURLs = member_menu_box[0].find_all("a")
    writerProfileBestURL = baseURL + writerProfileURLs[1]["href"]
    writerProfileBOBURL = baseURL + writerProfileURLs[2]["href"]

    writerProfileReq = urllib.request.Request(writerProfileBestURL,headers=hdr)
    writerProfileDoc = ""
    with urllib.request.urlopen(writerProfileReq) as url:
        writerProfileDoc = url.read()

    writerProfileSoup = BeautifulSoup(writerProfileDoc, "html.parser")    
    table_list = writerProfileSoup.find_all("table", class_="table_list")
    bestPostCount = table_list[0].find("a").text            # 유저가 쓴 베스트 개수

    writerProfileReq = urllib.request.Request(writerProfileBOBURL,headers=hdr)
    writerProfileDoc = ""
    with urllib.request.urlopen(writerProfileReq) as url:
        writerProfileDoc = url.read()

    writerProfileSoup = BeautifulSoup(writerProfileDoc, "html.parser")    
    table_list = writerProfileSoup.find_all("table", class_="table_list")
    BOBPostCount = table_list[0].find("a").text             # 유저가 쓴 베오베 개수

    ################################

    ## get Bulletin board Info
    viewSubjectDiv = soup.find_all("div", class_="viewSubjectDiv")

    board = viewSubjectDiv[0].find_all("span")[0]["class"][1]    # 게시판 이름 
    title = viewSubjectDiv[0].find_all("div")[0].text.strip()         # 제목

    ## post option : 창작글, 펌글, 베스트 금지, 베오베 금지, 본인삭제금지, 외부펌 금지
    contentContainer = soup.find_all("div", class_="contentContainer")
    postOption = contentContainer[0].find_all("table")

    changjakOption = False # 창작글
    permOption = False # 펌글
    prohibitBestOption = False # 베스트 금지
    prohibitBOBOption = False # 베오베 금지
    prohibitBoninOption = False # 본인삭제 금지
    prohibitOutsidePermOption = False # 외부펌 금지


    if len(postOption) is 2 :  # 첫 <table>은 글 옵션 두번째 <table>은 출처. 즉 테이블이 1개이면 출처만 있고 옵션이 없음

        options = postOption[0].find_all("li")

        for option in options : 
            optionName = option.find_all("div")[1].text
            if optionName == "창작글" :
                changjakOption = True
            elif optionName == "펌글" :
                permOption = True
            elif optionName == "베스트금지" :
                prohibitBestOption = True
            elif optionName == "베오베금지" :
                prohibitBOBOption = True
            elif optionName == "본인삭제금지" :
                prohibitBoninOption = True
            elif optionName == "외부펌금지" :
                prohibitOutsidePermOption = True    

    ## 본문 내용 가져오기
    # 구해야 할것 :  이미지 개수, 동영상 유무(youtube or 움짤), 텍스트 길이, 키워드

    viewContent = soup.find_all("div", class_="viewContent")
    imgs = viewContent[0].find_all("img") # 본문 삽입된 이미지들
    videos = viewContent[0].find_all("video")
    youtubes = viewContent[0].find_all("iframe")
    texts = viewContent[0].find_all("div")

    textData = []
    for text in texts :
        if (text.text) :
            textData.append(text.text)

    imgCount = len(imgs)   # 이미지 카운트
    videoCount = len(videos)   # 움짤 카운트
    youtubeCount = len(youtubes)   # 유투브(iframe) 개수
    textLineCount = len(textData)   # 텍스트 줄 수

    okList = getOkList(soup)
    postTime = datetime.strptime(postTime, '%Y/%m/%d %H:%M:%S')
    okTime = list(map(lambda x: (x - postTime).total_seconds(), okList))
    memoList = getMemoList(soup)
    memoTime = list(map(lambda x: (x - postTime).total_seconds(), memoList))

    import collections
    articleInfo = collections.OrderedDict.fromkeys(infos)
    for entity in infos:
        exec("articleInfo['{}'] = {}".format(entity, entity))

    return articleInfo


Overwriting getArticleInfos.py


In [ ]:
import pandas as pd
pd.DataFrame.from_dict(articleInfo,orient='index')